In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
data=pd.read_csv('LP-IV-datasets/ECGdataset(Ass4)/ecg_autoencoder_dataset.csv',header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818287,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490658,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1


In [5]:
features=data.drop(140,axis=1)
target=data[140]
xtrain,xtest,ytrain,ytest=train_test_split(features,target,test_size=0.2)
train_index=ytrain[ytrain==1].index
train_data=xtrain.loc[train_index]

In [6]:
minmaxscaler=MinMaxScaler()
x_train_scaled=minmaxscaler.fit_transform(train_data.copy())
x_test_scaled=minmaxscaler.transform(xtest.copy())

In [8]:
class AutoEncoder(Model):
    def __init__(self,output_units,ldim=8):
        super().__init__()
        self.encoder=Sequential([
            Dense(64,activation="relu"),
            Dropout(0.1),
            Dense(32,activation="relu"),
            Dropout(0.1),
            Dense(16,activation="relu"),
            Dropout(0.1),
            Dense(ldim,activation="relu")
        ])
        
        self.decoder=Sequential([
            Dense(16,activation="relu"),
            Dropout(0.1),
            Dense(32,activation="relu"),
            Dropout(0.1),
            Dense(64,activation="relu"),
            Dropout(0.1),
            Dense(output_units,activation="sigmoid")
        ])
    
    def call(self,input):
        encoded=self.encoder(input)
        decoded=self.decoder(encoded)
        return decoded
        

In [9]:
model=AutoEncoder(output_units=x_train_scaled.shape[1])
model.compile(optimizer="adam",loss='msle',metrics=['mse'])
model.fit(x_train_scaled,x_train_scaled,epochs=20,batch_size=512,validation_data=(x_test_scaled,x_test_scaled))

Epoch 1/20
5/5 [==============================] - 4s 149ms/step - loss: 0.0105 - mse: 0.0237 - val_loss: 0.0130 - val_mse: 0.0306
Epoch 2/20
5/5 [==============================] - 0s 51ms/step - loss: 0.0099 - mse: 0.0224 - val_loss: 0.0127 - val_mse: 0.0299
Epoch 3/20
5/5 [==============================] - 0s 55ms/step - loss: 0.0090 - mse: 0.0204 - val_loss: 0.0125 - val_mse: 0.0294
Epoch 4/20
5/5 [==============================] - 0s 53ms/step - loss: 0.0080 - mse: 0.0182 - val_loss: 0.0123 - val_mse: 0.0289
Epoch 5/20
5/5 [==============================] - 0s 55ms/step - loss: 0.0071 - mse: 0.0162 - val_loss: 0.0118 - val_mse: 0.0277
Epoch 6/20
5/5 [==============================] - 0s 46ms/step - loss: 0.0064 - mse: 0.0144 - val_loss: 0.0114 - val_mse: 0.0268
Epoch 7/20
5/5 [==============================] - 0s 46ms/step - loss: 0.0059 - mse: 0.0132 - val_loss: 0.0110 - val_mse: 0.0259
Epoch 8/20
5/5 [==============================] - 0s 53ms/step - loss: 0.0055 - mse: 0.0123 - va

In [10]:
def find_threshold(model,x_train_scaled):
    reconstruction=model.predict(x_train_scaled)
    re_err=tf.keras.losses.msle(reconstruction,x_train_scaled)
    threshold=np.mean(re_err.numpy())+np.std(re_err.numpy())
    return threshold
def get_predictions(model,x_test_scaled,threshold):
    reconstruction=model.predict(x_test_scaled)
    error=tf.keras.losses.msle(reconstruction,x_test_scaled)
    anomaly_mask=pd.Series(error)>threshold
    pred=anomaly_mask.map(lambda x:0.0 if x==True else 1.0)
    return pred

In [11]:
threshold=find_threshold(model,x_train_scaled)
threshold

74/74 [==============================] - 1s 4ms/step


0.009564119877452884

In [12]:
pred=get_predictions(model,x_test_scaled,threshold)
acc=accuracy_score(pred,ytest)
acc

32/32 [==============================] - 0s 4ms/step


0.943